In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 226.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 166.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 137.6 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import torch

print("PyTorch version:", torch.__version__)


PyTorch version: 2.1.0+cu118


In [3]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [4]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
print(device)

cuda


In [6]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [7]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.

In [8]:

model_path = 'Best_Teacher.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [9]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [10]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [11]:
# Logits normalization function
def normalize(logit):
    mean = logit.mean(dim=-1, keepdim=True)
    stdv = logit.std(dim=-1, keepdim=True)
    return (logit - mean) / (1e-7 + stdv)


In [12]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [13]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [14]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [15]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [17]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [37]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and len(param.shape) == 4:  # Conv weights only
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    # Add momentum for gradient accumulation smoothing
    momentum = 0.9  # Controls exponential moving average
    accumulated_batches = 0  # Track for bias correction

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature


            # Compute losses
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss

            # Modified backward propagation
            loss.backward()

            # Accumulate importance scores with parameter-gradient product
            accumulated_batches += 1
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    # Key modification: Use parameter-gradient product magnitude
                    grad_product = (param.data * param.grad).abs_()
                    
                    # Exponential moving average with bias correction
                    if accumulated_batches == 1:
                        importance_scores[name] = grad_product
                    else:
                        importance_scores[name] = momentum * importance_scores[name] + (1 - momentum) * grad_product

    # Apply bias correction for EMA
    for name in importance_scores:
        importance_scores[name] /= (1 - momentum**accumulated_batches)

    return importance_scores

In [19]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [20]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)


            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [22]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD and logits normalization
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50, temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Temperature scaling
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # Logits normalization
            teacher_logits_temp = normalize(teacher_logits_temp)
            student_logits_temp = normalize(student_logits_temp)

            # CA-KLD loss (normalized logits)
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [21]:

student = train_kd_pruning(
    teacher, student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.5,beta_prob=0.5, patience=5,save_path="student_before_pruning.pth"
)


Epoch 1/50 | Train Loss: 2.6007 | Train Acc: 83.86% | Val Acc: 88.68%
Epoch 2/50 | Train Loss: 0.8082 | Train Acc: 94.44% | Val Acc: 92.16%
Epoch 3/50 | Train Loss: 0.4497 | Train Acc: 97.41% | Val Acc: 94.04%
Epoch 4/50 | Train Loss: 0.2881 | Train Acc: 98.71% | Val Acc: 94.84%
Epoch 5/50 | Train Loss: 0.2302 | Train Acc: 99.09% | Val Acc: 95.51%
Epoch 6/50 | Train Loss: 0.1968 | Train Acc: 99.22% | Val Acc: 95.55%
Epoch 7/50 | Train Loss: 0.1769 | Train Acc: 99.28% | Val Acc: 95.57%
Epoch 8/50 | Train Loss: 0.1595 | Train Acc: 99.32% | Val Acc: 95.84%
Epoch 9/50 | Train Loss: 0.1477 | Train Acc: 99.35% | Val Acc: 96.00%
Epoch 10/50 | Train Loss: 0.1395 | Train Acc: 99.33% | Val Acc: 95.79%
Epoch 11/50 | Train Loss: 0.1308 | Train Acc: 99.36% | Val Acc: 96.12%
Epoch 12/50 | Train Loss: 0.1276 | Train Acc: 99.30% | Val Acc: 95.96%
Epoch 13/50 | Train Loss: 0.1185 | Train Acc: 99.39% | Val Acc: 96.05%
Epoch 14/50 | Train Loss: 0.1156 | Train Acc: 99.32% | Val Acc: 96.18%
Epoch 15/50 | T

In [24]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.40%
Student Model Test Accuracy Before Pruning: 95.92%


## 93% Sparsity

In [37]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [38]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 41s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [39]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")


Student Model Test Accuracy After Pruning: 10.01%


In [40]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 92.16%
Epoch 1/200 | Train Loss: 0.3085 | Train Acc: 90.44%
Validation Loss: 0.2557 | Validation Acc: 92.16% | Sparsity: 93.92%

New best model saved with Val Accuracy: 92.25%
Epoch 2/200 | Train Loss: 0.1190 | Train Acc: 96.02%
Validation Loss: 0.2487 | Validation Acc: 92.25% | Sparsity: 93.92%

New best model saved with Val Accuracy: 92.75%
Epoch 3/200 | Train Loss: 0.0681 | Train Acc: 97.79%
Validation Loss: 0.2521 | Validation Acc: 92.75% | Sparsity: 93.92%

Epoch 4/200 | Train Loss: 0.0408 | Train Acc: 98.77%
Validation Loss: 0.2707 | Validation Acc: 92.47% | Sparsity: 93.92%

Epoch 5/200 | Train Loss: 0.0235 | Train Acc: 99.33%
Validation Loss: 0.2756 | Validation Acc: 92.61% | Sparsity: 93.92%

Epoch 6/200 | Train Loss: 0.0138 | Train Acc: 99.69%
Validation Loss: 0.2987 | Validation Acc: 92.55% | Sparsity: 93.92%

New best model saved with Val Accuracy: 92.83%
Epoch 7/200 | Train Loss: 0.0099 | Train Acc: 99.78%
Validation Loss: 0.3030 | V

In [41]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 92.97%


In [109]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [110]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [111]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5364 | Train Acc: 92.82% | Val Loss: 0.1733 | Val Acc: 94.47% | Sparsity: 93.92%
Epoch 2/50 | Train Loss: 0.2549 | Train Acc: 96.94% | Val Loss: 0.1414 | Val Acc: 95.69% | Sparsity: 93.92%
Epoch 3/50 | Train Loss: 0.1832 | Train Acc: 98.16% | Val Loss: 0.1529 | Val Acc: 95.37% | Sparsity: 93.92%
Epoch 4/50 | Train Loss: 0.1452 | Train Acc: 98.83% | Val Loss: 0.1199 | Val Acc: 96.20% | Sparsity: 93.92%
Epoch 5/50 | Train Loss: 0.1244 | Train Acc: 99.08% | Val Loss: 0.1234 | Val Acc: 96.14% | Sparsity: 93.92%
Epoch 6/50 | Train Loss: 0.1141 | Train Acc: 99.14% | Val Loss: 0.1238 | Val Acc: 96.17% | Sparsity: 93.92%
Epoch 7/50 | Train Loss: 0.1071 | Train Acc: 99.20% | Val Loss: 0.1167 | Val Acc: 96.38% | Sparsity: 93.92%
Epoch 8/50 | Train Loss: 0.1016 | Train Acc: 99.27% | Val Loss: 0.1148 | Val Acc: 96.34% | Sparsity: 93.92%
Epoch 9/50 | Train Loss: 0.0959 | Train Acc: 99.28% | Val Loss: 0.1198 | Val Acc: 96.16% | Sparsity: 93.92%
Epoch 10/50 | Train Loss: 0.

In [48]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 94.28%


In [49]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [50]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 39s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [51]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.8395 | Train Acc: 90.09% | Val Loss: 0.3916 | Val Acc: 90.28% | Sparsity: 93.92%
Epoch 2/50 | Train Loss: 0.9744 | Train Acc: 94.80% | Val Loss: 0.3487 | Val Acc: 91.18% | Sparsity: 93.92%
Epoch 3/50 | Train Loss: 0.6462 | Train Acc: 96.96% | Val Loss: 0.3394 | Val Acc: 91.62% | Sparsity: 93.92%
Epoch 4/50 | Train Loss: 0.4943 | Train Acc: 97.92% | Val Loss: 0.2662 | Val Acc: 93.17% | Sparsity: 93.92%
Epoch 5/50 | Train Loss: 0.4059 | Train Acc: 98.45% | Val Loss: 0.2648 | Val Acc: 93.47% | Sparsity: 93.92%
Epoch 6/50 | Train Loss: 0.3360 | Train Acc: 98.85% | Val Loss: 0.2153 | Val Acc: 94.02% | Sparsity: 93.92%
Epoch 7/50 | Train Loss: 0.2976 | Train Acc: 98.92% | Val Loss: 0.2189 | Val Acc: 94.21% | Sparsity: 93.92%
Epoch 8/50 | Train Loss: 0.2742 | Train Acc: 98.96% | Val Loss: 0.2103 | Val Acc: 94.17% | Sparsity: 93.92%
Epoch 9/50 | Train Loss: 0.2506 | Train Acc: 99.08% | Val Loss: 0.2095 | Val Acc: 94.39% | Sparsity: 93.92%
Epoch 10/50 | Train Loss: 0.

In [52]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 94.40%


# 97.7 %

In [25]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [26]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9779)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 20s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [29]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

Sparsity Before Pruning: 97.70%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

In [42]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [43]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9779)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 20s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 3.9455 | Train Acc: 79.70% | Val Loss: 0.5662 | Val Acc: 85.19% | Sparsity: 97.70%
Epoch 2/50 | Train Loss: 1.9935 | Train Acc: 89.17% | Val Loss: 0.4604 | Val Acc: 87.81% | Sparsity: 97.70%
Epoch 3/50 | Train Loss: 1.5562 | Train Acc: 91.60% | Val Loss: 0.4667 | Val Acc: 88.36% | Sparsity: 97.70%
Epoch 4/50 | Train Loss: 1.2685 | Train Acc: 93.18% | Val Loss: 0.3882 | Val Acc: 90.05% | Sparsity: 97.70%
Epoch 5/50 | Train Loss: 1.0350 | Train Acc: 94.63% | Val Loss: 0.4008 | Val Acc: 89.97% | Sparsity: 97.70%
Epoch 6/50 | Train Loss: 0.8861 | Train Acc: 95.70% | Val Loss: 0.3448 | Val Acc: 91.32% | Sparsity: 97.70%
Epoch 7/50 | Train Loss: 0.7841 | Train Acc: 96.37% | Val Loss: 0.3573 | Val Acc: 90.86% | Sparsity: 97.70%
Epoch 8/50 | Train Loss: 0.6875 | Train Acc: 96.89% | Val Loss: 0.2956 | Val Acc: 92.12% | Sparsity: 97.70%
Epoch 9/50 | Train Loss: 0.6120 | Train Acc: 97.36% | Val Loss: 0.3238 | Val Acc: 90.97% | Sparsity: 97.70%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

In [38]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [39]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9779)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 18s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [40]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.4256 | Train Acc: 81.53% | Val Loss: 0.4337 | Val Acc: 86.70% | Sparsity: 97.70%
Epoch 2/50 | Train Loss: 0.6068 | Train Acc: 91.63% | Val Loss: 0.3076 | Val Acc: 90.63% | Sparsity: 97.70%
Epoch 3/50 | Train Loss: 0.4503 | Train Acc: 94.19% | Val Loss: 0.3068 | Val Acc: 91.07% | Sparsity: 97.70%
Epoch 4/50 | Train Loss: 0.3710 | Train Acc: 95.13% | Val Loss: 0.2788 | Val Acc: 91.44% | Sparsity: 97.70%
Epoch 5/50 | Train Loss: 0.3046 | Train Acc: 96.40% | Val Loss: 0.2551 | Val Acc: 92.38% | Sparsity: 97.70%
Epoch 6/50 | Train Loss: 0.2502 | Train Acc: 97.30% | Val Loss: 0.2291 | Val Acc: 92.75% | Sparsity: 97.70%
Epoch 7/50 | Train Loss: 0.2216 | Train Acc: 97.88% | Val Loss: 0.2494 | Val Acc: 92.40% | Sparsity: 97.70%
Epoch 8/50 | Train Loss: 0.1918 | Train Acc: 98.30% | Val Loss: 0.2584 | Val Acc: 92.14% | Sparsity: 97.70%
Epoch 9/50 | Train Loss: 0.1832 | Train Acc: 98.44% | Val Loss: 0.2586 | Val Acc: 92.54% | Sparsity: 97.70%
Epoch 10/50 | Train Loss: 0.

In [41]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 91.29%


# 98% of Sparsity

In [31]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [32]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [33]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=7
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 76.94%
Epoch 1/200 | Train Loss: 1.2940 | Train Acc: 54.71%
Validation Loss: 0.6902 | Validation Acc: 76.94% | Sparsity: 98.41%

New best model saved with Val Accuracy: 83.55%
Epoch 2/200 | Train Loss: 0.5647 | Train Acc: 80.83%
Validation Loss: 0.4949 | Validation Acc: 83.55% | Sparsity: 98.41%

New best model saved with Val Accuracy: 85.09%
Epoch 3/200 | Train Loss: 0.4149 | Train Acc: 85.82%
Validation Loss: 0.4345 | Validation Acc: 85.09% | Sparsity: 98.41%

New best model saved with Val Accuracy: 86.22%
Epoch 4/200 | Train Loss: 0.3286 | Train Acc: 88.81%
Validation Loss: 0.4003 | Validation Acc: 86.22% | Sparsity: 98.41%

New best model saved with Val Accuracy: 86.95%
Epoch 5/200 | Train Loss: 0.2701 | Train Acc: 90.81%
Validation Loss: 0.3871 | Validation Acc: 86.95% | Sparsity: 98.41%

New best model saved with Val Accuracy: 87.27%
Epoch 6/200 | Train Loss: 0.2230 | Train Acc: 92.40%
Validation Loss: 0.3859 | Validation Acc: 87.27% | Spar

In [34]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 86.99%


In [87]:

# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


Pruned Student Model Inference Time: 1.65 ms per batch


# Retrained with KD

In [35]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [36]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 95.92%


In [37]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [38]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 5.4962 | Train Acc: 72.37% | Val Loss: 1.0768 | Val Acc: 77.60% | Sparsity: 98.41%
Epoch 2/50 | Train Loss: 2.6102 | Train Acc: 85.92% | Val Loss: 0.9028 | Val Acc: 81.48% | Sparsity: 98.41%
Epoch 3/50 | Train Loss: 2.0182 | Train Acc: 89.03% | Val Loss: 0.6907 | Val Acc: 84.25% | Sparsity: 98.41%
Epoch 4/50 | Train Loss: 1.6901 | Train Acc: 90.86% | Val Loss: 0.5981 | Val Acc: 86.98% | Sparsity: 98.41%
Epoch 5/50 | Train Loss: 1.4845 | Train Acc: 92.20% | Val Loss: 0.5247 | Val Acc: 87.75% | Sparsity: 98.41%
Epoch 6/50 | Train Loss: 1.2893 | Train Acc: 93.29% | Val Loss: 0.5562 | Val Acc: 87.50% | Sparsity: 98.41%
Epoch 7/50 | Train Loss: 1.1821 | Train Acc: 94.02% | Val Loss: 0.5146 | Val Acc: 88.33% | Sparsity: 98.41%
Epoch 8/50 | Train Loss: 1.0158 | Train Acc: 94.86% | Val Loss: 0.4488 | Val Acc: 89.03% | Sparsity: 98.41%
Epoch 9/50 | Train Loss: 0.9443 | Train Acc: 95.42% | Val Loss: 0.5284 | Val Acc: 88.04% | Sparsity: 98.41%
Epoch 10/50 | Train Loss: 0.

In [39]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 90.66%


In [47]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [48]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=6.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [49]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=6.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 7.8585 | Train Acc: 71.83% | Val Loss: 1.0331 | Val Acc: 80.50% | Sparsity: 98.41%
Epoch 2/50 | Train Loss: 3.9110 | Train Acc: 85.05% | Val Loss: 1.1179 | Val Acc: 80.87% | Sparsity: 98.41%
Epoch 3/50 | Train Loss: 3.0821 | Train Acc: 88.15% | Val Loss: 1.1168 | Val Acc: 80.55% | Sparsity: 98.41%
Epoch 4/50 | Train Loss: 2.6224 | Train Acc: 89.91% | Val Loss: 0.7715 | Val Acc: 84.41% | Sparsity: 98.41%
Epoch 5/50 | Train Loss: 2.2793 | Train Acc: 91.24% | Val Loss: 0.5782 | Val Acc: 87.38% | Sparsity: 98.41%
Epoch 6/50 | Train Loss: 1.9656 | Train Acc: 92.64% | Val Loss: 0.5399 | Val Acc: 87.86% | Sparsity: 98.41%
Epoch 7/50 | Train Loss: 1.7270 | Train Acc: 93.53% | Val Loss: 0.7195 | Val Acc: 86.16% | Sparsity: 98.41%
Epoch 8/50 | Train Loss: 1.5379 | Train Acc: 94.57% | Val Loss: 0.4978 | Val Acc: 89.39% | Sparsity: 98.41%
Epoch 9/50 | Train Loss: 1.4292 | Train Acc: 94.94% | Val Loss: 0.5514 | Val Acc: 88.51% | Sparsity: 98.41%
Epoch 10/50 | Train Loss: 1.

In [50]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 90.42%


In [53]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [54]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.985)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [55]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=7,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.9873 | Train Acc: 74.02% | Val Loss: 0.4614 | Val Acc: 85.34% | Sparsity: 98.41%
Epoch 2/50 | Train Loss: 0.8371 | Train Acc: 88.41% | Val Loss: 0.3861 | Val Acc: 87.88% | Sparsity: 98.41%
Epoch 3/50 | Train Loss: 0.6410 | Train Acc: 91.30% | Val Loss: 0.3666 | Val Acc: 88.96% | Sparsity: 98.41%
Epoch 4/50 | Train Loss: 0.5248 | Train Acc: 93.02% | Val Loss: 0.3447 | Val Acc: 89.98% | Sparsity: 98.41%
Epoch 5/50 | Train Loss: 0.4446 | Train Acc: 94.37% | Val Loss: 0.3315 | Val Acc: 90.31% | Sparsity: 98.41%
Epoch 6/50 | Train Loss: 0.3928 | Train Acc: 95.14% | Val Loss: 0.3643 | Val Acc: 89.18% | Sparsity: 98.41%
Epoch 7/50 | Train Loss: 0.3485 | Train Acc: 95.89% | Val Loss: 0.3765 | Val Acc: 89.14% | Sparsity: 98.41%
Epoch 8/50 | Train Loss: 0.3047 | Train Acc: 96.66% | Val Loss: 0.3309 | Val Acc: 90.34% | Sparsity: 98.41%
Epoch 9/50 | Train Loss: 0.2772 | Train Acc: 97.04% | Val Loss: 0.3258 | Val Acc: 90.49% | Sparsity: 98.41%
Epoch 10/50 | Train Loss: 0.

In [56]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 90.79%


## 90% Sparsity

In [57]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [58]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [59]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 96.45%
Epoch 1/100 | Train Loss: 0.1748 | Train Acc: 94.61%
Validation Loss: 0.1067 | Validation Acc: 96.45% | Sparsity: 90.00%

New best model saved with Val Accuracy: 96.63%
Epoch 2/100 | Train Loss: 0.0704 | Train Acc: 97.73%
Validation Loss: 0.1004 | Validation Acc: 96.63% | Sparsity: 90.00%

New best model saved with Val Accuracy: 96.65%
Epoch 3/100 | Train Loss: 0.0347 | Train Acc: 99.03%
Validation Loss: 0.1018 | Validation Acc: 96.65% | Sparsity: 90.00%

New best model saved with Val Accuracy: 96.83%
Epoch 4/100 | Train Loss: 0.0156 | Train Acc: 99.65%
Validation Loss: 0.1068 | Validation Acc: 96.83% | Sparsity: 90.00%

New best model saved with Val Accuracy: 96.88%
Epoch 5/100 | Train Loss: 0.0091 | Train Acc: 99.82%
Validation Loss: 0.1078 | Validation Acc: 96.88% | Sparsity: 90.00%

New best model saved with Val Accuracy: 97.04%
Epoch 6/100 | Train Loss: 0.0056 | Train Acc: 99.92%
Validation Loss: 0.1033 | Validation Acc: 97.04% | Spar

In [60]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 94.12%


In [61]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [62]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [63]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.2966 | Train Acc: 93.03% | Val Loss: 0.2835 | Val Acc: 92.06% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 0.7213 | Train Acc: 96.45% | Val Loss: 0.2121 | Val Acc: 93.90% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 0.4764 | Train Acc: 97.99% | Val Loss: 0.1292 | Val Acc: 96.30% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 0.3373 | Train Acc: 98.78% | Val Loss: 0.1185 | Val Acc: 96.52% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 0.2940 | Train Acc: 98.89% | Val Loss: 0.1035 | Val Acc: 96.91% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 0.2517 | Train Acc: 99.07% | Val Loss: 0.1107 | Val Acc: 96.72% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 0.2314 | Train Acc: 99.11% | Val Loss: 0.1008 | Val Acc: 97.02% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 0.2172 | Train Acc: 99.11% | Val Loss: 0.0999 | Val Acc: 97.10% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 0.2021 | Train Acc: 99.12% | Val Loss: 0.0945 | Val Acc: 97.04% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [64]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 94.83%


In [25]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [26]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=6.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 41s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [27]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy: 10.06%


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.3757 | Train Acc: 95.11% | Val Loss: 0.1225 | Val Acc: 96.02% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 0.1820 | Train Acc: 98.07% | Val Loss: 0.1027 | Val Acc: 96.78% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 0.1372 | Train Acc: 98.83% | Val Loss: 0.0904 | Val Acc: 97.29% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 0.1129 | Train Acc: 99.18% | Val Loss: 0.0895 | Val Acc: 97.31% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 0.1008 | Train Acc: 99.23% | Val Loss: 0.0855 | Val Acc: 97.40% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 0.0943 | Train Acc: 99.30% | Val Loss: 0.0893 | Val Acc: 97.41% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 0.0881 | Train Acc: 99.28% | Val Loss: 0.0812 | Val Acc: 97.56% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 0.0831 | Train Acc: 99.36% | Val Loss: 0.0803 | Val Acc: 97.63% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 0.0798 | Train Acc: 99.34% | Val Loss: 0.0799 | Val Acc: 97.46% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

# 80% Sparsity

In [68]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [69]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [70]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 17.07%


In [71]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 98.16%
Epoch 1/100 | Train Loss: 0.0727 | Train Acc: 97.86%
Validation Loss: 0.0559 | Validation Acc: 98.16% | Sparsity: 80.00%

New best model saved with Val Accuracy: 98.22%
Epoch 2/100 | Train Loss: 0.0274 | Train Acc: 99.19%
Validation Loss: 0.0548 | Validation Acc: 98.22% | Sparsity: 80.00%

New best model saved with Val Accuracy: 98.33%
Epoch 3/100 | Train Loss: 0.0101 | Train Acc: 99.78%
Validation Loss: 0.0541 | Validation Acc: 98.33% | Sparsity: 80.00%

New best model saved with Val Accuracy: 98.40%
Epoch 4/100 | Train Loss: 0.0045 | Train Acc: 99.95%
Validation Loss: 0.0517 | Validation Acc: 98.40% | Sparsity: 80.00%

New best model saved with Val Accuracy: 98.45%
Epoch 5/100 | Train Loss: 0.0027 | Train Acc: 99.97%
Validation Loss: 0.0528 | Validation Acc: 98.45% | Sparsity: 80.00%

Epoch 6/100 | Train Loss: 0.0021 | Train Acc: 99.98%
Validation Loss: 0.0536 | Validation Acc: 98.43% | Sparsity: 80.00%

Epoch 7/100 | Train Loss: 0.0014 

In [72]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 95.21%


In [73]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [74]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [75]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [76]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.6830 | Train Acc: 96.57% | Val Loss: 0.1491 | Val Acc: 95.92% | Sparsity: 80.00%
Epoch 2/50 | Train Loss: 0.4355 | Train Acc: 98.10% | Val Loss: 0.1026 | Val Acc: 97.06% | Sparsity: 80.00%
Epoch 3/50 | Train Loss: 0.2937 | Train Acc: 98.87% | Val Loss: 0.0668 | Val Acc: 98.07% | Sparsity: 80.00%
Epoch 4/50 | Train Loss: 0.2330 | Train Acc: 99.08% | Val Loss: 0.0644 | Val Acc: 98.09% | Sparsity: 80.00%
Epoch 5/50 | Train Loss: 0.2067 | Train Acc: 99.11% | Val Loss: 0.0643 | Val Acc: 98.06% | Sparsity: 80.00%
Epoch 6/50 | Train Loss: 0.1854 | Train Acc: 99.13% | Val Loss: 0.0623 | Val Acc: 98.07% | Sparsity: 80.00%
Epoch 7/50 | Train Loss: 0.1715 | Train Acc: 99.12% | Val Loss: 0.0634 | Val Acc: 98.12% | Sparsity: 80.00%
Epoch 8/50 | Train Loss: 0.1630 | Train Acc: 99.12% | Val Loss: 0.0555 | Val Acc: 98.27% | Sparsity: 80.00%
Epoch 9/50 | Train Loss: 0.1569 | Train Acc: 99.19% | Val Loss: 0.0572 | Val Acc: 98.35% | Sparsity: 80.00%
Epoch 10/50 | Train Loss: 0.

In [77]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 95.49%


In [78]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [79]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 37s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [80]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.1684 | Train Acc: 98.07% | Val Loss: 0.0553 | Val Acc: 98.37% | Sparsity: 80.00%
Epoch 2/50 | Train Loss: 0.1043 | Train Acc: 99.03% | Val Loss: 0.0508 | Val Acc: 98.42% | Sparsity: 80.00%
Epoch 3/50 | Train Loss: 0.0839 | Train Acc: 99.27% | Val Loss: 0.0481 | Val Acc: 98.49% | Sparsity: 80.00%
Epoch 4/50 | Train Loss: 0.0746 | Train Acc: 99.40% | Val Loss: 0.0475 | Val Acc: 98.50% | Sparsity: 80.00%
Epoch 5/50 | Train Loss: 0.0707 | Train Acc: 99.33% | Val Loss: 0.0469 | Val Acc: 98.52% | Sparsity: 80.00%
Epoch 6/50 | Train Loss: 0.0669 | Train Acc: 99.33% | Val Loss: 0.0457 | Val Acc: 98.61% | Sparsity: 80.00%
Epoch 7/50 | Train Loss: 0.0645 | Train Acc: 99.43% | Val Loss: 0.0480 | Val Acc: 98.49% | Sparsity: 80.00%
Epoch 8/50 | Train Loss: 0.0608 | Train Acc: 99.41% | Val Loss: 0.0456 | Val Acc: 98.58% | Sparsity: 80.00%
Epoch 9/50 | Train Loss: 0.0591 | Train Acc: 99.38% | Val Loss: 0.0454 | Val Acc: 98.56% | Sparsity: 80.00%
Epoch 10/50 | Train Loss: 0.

In [81]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 95.44%


# 75% of Sparsity

In [82]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [83]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [84]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 23.03%


In [85]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 98.45%
Epoch 1/100 | Train Loss: 0.0620 | Train Acc: 98.09%
Validation Loss: 0.0466 | Validation Acc: 98.45% | Sparsity: 75.00%

New best model saved with Val Accuracy: 98.58%
Epoch 2/100 | Train Loss: 0.0215 | Train Acc: 99.38%
Validation Loss: 0.0469 | Validation Acc: 98.58% | Sparsity: 75.00%

New best model saved with Val Accuracy: 98.60%
Epoch 3/100 | Train Loss: 0.0079 | Train Acc: 99.83%
Validation Loss: 0.0443 | Validation Acc: 98.60% | Sparsity: 75.00%

New best model saved with Val Accuracy: 98.65%
Epoch 4/100 | Train Loss: 0.0036 | Train Acc: 99.94%
Validation Loss: 0.0441 | Validation Acc: 98.65% | Sparsity: 75.00%

Epoch 5/100 | Train Loss: 0.0022 | Train Acc: 99.98%
Validation Loss: 0.0460 | Validation Acc: 98.58% | Sparsity: 75.00%

New best model saved with Val Accuracy: 98.67%
Epoch 6/100 | Train Loss: 0.0016 | Train Acc: 99.99%
Validation Loss: 0.0462 | Validation Acc: 98.67% | Sparsity: 75.00%

Epoch 7/100 | Train Loss: 0.0010 

In [86]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.46%


In [121]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 1.70 ms per batch


# Retrain with KD

In [87]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [88]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [89]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5216 | Train Acc: 97.53% | Val Loss: 0.1000 | Val Acc: 97.08% | Sparsity: 75.00%
Epoch 2/50 | Train Loss: 0.3363 | Train Acc: 98.57% | Val Loss: 0.0746 | Val Acc: 97.80% | Sparsity: 75.00%
Epoch 3/50 | Train Loss: 0.2519 | Train Acc: 98.95% | Val Loss: 0.0639 | Val Acc: 98.30% | Sparsity: 75.00%
Epoch 4/50 | Train Loss: 0.2074 | Train Acc: 99.11% | Val Loss: 0.0507 | Val Acc: 98.45% | Sparsity: 75.00%
Epoch 5/50 | Train Loss: 0.1873 | Train Acc: 99.12% | Val Loss: 0.0530 | Val Acc: 98.45% | Sparsity: 75.00%
Epoch 6/50 | Train Loss: 0.1716 | Train Acc: 99.08% | Val Loss: 0.0490 | Val Acc: 98.54% | Sparsity: 75.00%
Epoch 7/50 | Train Loss: 0.1591 | Train Acc: 99.17% | Val Loss: 0.0535 | Val Acc: 98.46% | Sparsity: 75.00%
Epoch 8/50 | Train Loss: 0.1515 | Train Acc: 99.16% | Val Loss: 0.0515 | Val Acc: 98.38% | Sparsity: 75.00%
Epoch 9/50 | Train Loss: 0.1414 | Train Acc: 99.14% | Val Loss: 0.0503 | Val Acc: 98.46% | Sparsity: 75.00%
Epoch 10/50 | Train Loss: 0.

In [90]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.60%


In [91]:
# # Measure inference times
# pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
# print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

In [92]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [93]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [94]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.1453 | Train Acc: 98.37% | Val Loss: 0.0470 | Val Acc: 98.57% | Sparsity: 75.00%
Epoch 2/50 | Train Loss: 0.0927 | Train Acc: 99.19% | Val Loss: 0.0435 | Val Acc: 98.66% | Sparsity: 75.00%
Epoch 3/50 | Train Loss: 0.0766 | Train Acc: 99.31% | Val Loss: 0.0433 | Val Acc: 98.66% | Sparsity: 75.00%
Epoch 4/50 | Train Loss: 0.0693 | Train Acc: 99.34% | Val Loss: 0.0420 | Val Acc: 98.73% | Sparsity: 75.00%
Epoch 5/50 | Train Loss: 0.0651 | Train Acc: 99.41% | Val Loss: 0.0422 | Val Acc: 98.62% | Sparsity: 75.00%
Epoch 6/50 | Train Loss: 0.0610 | Train Acc: 99.38% | Val Loss: 0.0418 | Val Acc: 98.71% | Sparsity: 75.00%
Epoch 7/50 | Train Loss: 0.0586 | Train Acc: 99.43% | Val Loss: 0.0423 | Val Acc: 98.69% | Sparsity: 75.00%
Epoch 8/50 | Train Loss: 0.0554 | Train Acc: 99.40% | Val Loss: 0.0404 | Val Acc: 98.68% | Sparsity: 75.00%
Epoch 9/50 | Train Loss: 0.0546 | Train Acc: 99.41% | Val Loss: 0.0431 | Val Acc: 98.72% | Sparsity: 75.00%
Early stopping triggered at 

In [95]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.62%


## 50% Sparsity

In [96]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [97]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 36s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [98]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 91.82%


In [99]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 98.71%
Epoch 1/200 | Train Loss: 0.0433 | Train Acc: 98.68%
Validation Loss: 0.0402 | Validation Acc: 98.71% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.81%
Epoch 2/200 | Train Loss: 0.0126 | Train Acc: 99.66%
Validation Loss: 0.0383 | Validation Acc: 98.81% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.85%
Epoch 3/200 | Train Loss: 0.0038 | Train Acc: 99.96%
Validation Loss: 0.0365 | Validation Acc: 98.85% | Sparsity: 50.46%

Epoch 4/200 | Train Loss: 0.0018 | Train Acc: 99.99%
Validation Loss: 0.0383 | Validation Acc: 98.81% | Sparsity: 50.46%

Epoch 5/200 | Train Loss: 0.0012 | Train Acc: 99.99%
Validation Loss: 0.0380 | Validation Acc: 98.79% | Sparsity: 50.46%

Epoch 6/200 | Train Loss: 0.0008 | Train Acc: 100.00%
Validation Loss: 0.0384 | Validation Acc: 98.81% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.93%
Epoch 7/200 | Train Loss: 0.0006 | Train Acc: 100.00%
Validation Loss: 0.0377 |

In [100]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.06%


In [101]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [102]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 35s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [103]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.3536 | Train Acc: 98.38% | Val Loss: 0.0667 | Val Acc: 97.91% | Sparsity: 50.46%
Epoch 2/50 | Train Loss: 0.2323 | Train Acc: 98.94% | Val Loss: 0.0461 | Val Acc: 98.65% | Sparsity: 50.46%
Epoch 3/50 | Train Loss: 0.1863 | Train Acc: 99.09% | Val Loss: 0.0446 | Val Acc: 98.72% | Sparsity: 50.46%
Epoch 4/50 | Train Loss: 0.1591 | Train Acc: 99.12% | Val Loss: 0.0422 | Val Acc: 98.66% | Sparsity: 50.46%
Epoch 5/50 | Train Loss: 0.1432 | Train Acc: 99.16% | Val Loss: 0.0409 | Val Acc: 98.75% | Sparsity: 50.46%
Epoch 6/50 | Train Loss: 0.1335 | Train Acc: 99.19% | Val Loss: 0.0421 | Val Acc: 98.66% | Sparsity: 50.46%
Epoch 7/50 | Train Loss: 0.1259 | Train Acc: 99.19% | Val Loss: 0.0416 | Val Acc: 98.69% | Sparsity: 50.46%
Epoch 8/50 | Train Loss: 0.1181 | Train Acc: 99.18% | Val Loss: 0.0409 | Val Acc: 98.68% | Sparsity: 50.46%
Epoch 9/50 | Train Loss: 0.1137 | Train Acc: 99.15% | Val Loss: 0.0417 | Val Acc: 98.58% | Sparsity: 50.46%
Epoch 10/50 | Train Loss: 0.

In [104]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.03%


In [105]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [106]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=3.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 38s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [107]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=3.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.1033 | Train Acc: 98.71% | Val Loss: 0.0400 | Val Acc: 98.77% | Sparsity: 50.46%
Epoch 2/50 | Train Loss: 0.0678 | Train Acc: 99.31% | Val Loss: 0.0387 | Val Acc: 98.79% | Sparsity: 50.46%
Epoch 3/50 | Train Loss: 0.0575 | Train Acc: 99.40% | Val Loss: 0.0382 | Val Acc: 98.81% | Sparsity: 50.46%
Epoch 4/50 | Train Loss: 0.0535 | Train Acc: 99.42% | Val Loss: 0.0370 | Val Acc: 98.78% | Sparsity: 50.46%
Epoch 5/50 | Train Loss: 0.0501 | Train Acc: 99.45% | Val Loss: 0.0381 | Val Acc: 98.70% | Sparsity: 50.46%
Epoch 6/50 | Train Loss: 0.0486 | Train Acc: 99.36% | Val Loss: 0.0374 | Val Acc: 98.80% | Sparsity: 50.46%
Epoch 7/50 | Train Loss: 0.0467 | Train Acc: 99.43% | Val Loss: 0.0368 | Val Acc: 98.79% | Sparsity: 50.46%
Epoch 8/50 | Train Loss: 0.0446 | Train Acc: 99.47% | Val Loss: 0.0378 | Val Acc: 98.70% | Sparsity: 50.46%
Early stopping triggered at epoch 8. No improvement for 5 epochs.
Student model saved before pruning at: pruned_student_retrain_KD_90%.p

In [108]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 96.08%
